# Partie 1: Extraction des tweets 
``REMARQUE: Le code source de cette étape doit étre executer si seulement vous aimerai traiter le code from Scratch``

Pour faire du machine learning sur des tweets il faut des tweets. 

Pour cela on va utiliser l’API que Twitter lui-même propose. 

Elle permet de récupérer des tweets en ajoutant certaines conditions sur le type de tweets que nous souhaitons

J’ai choisi de récupérer les tweets sur trois fois qui contiennent les ***news*** ou ***actualités***

Les tweets doivent être en français et anglais 

voici comment récupérer des tweets avec **Tweepy**: 

1.	D’abord nous aurons besoin d’un compte Twitter développeur. 
2.	Une fois notre compte développeur crée, il vous faudra quelques informations : l’API key, l’API secret key, l’Access token, l’Access token secret. 
3.	Enfin, on peut extraire  les tweets



**`!pip install python-twitter`**


### 1.1   Authetifacation

In [1]:
import twitter

api = twitter.Api(consumer_key="FHolBw3mVFGLtbfLb1m8zGVFV",
                  consumer_secret="dIT8PT37qcpZkbaV1gnHfCiSHq60B9OmJnfxGQwwGyHacQ2n2T",
                  access_token_key="1334257369027579907-CrBHS22plIxU4wpSHaRR1wtZdNuKYv",
                  access_token_secret="Mga6h9zLXCZB53CPtDmbEAzprF8saZSI44Fis9ZfV7ooI")

**`!pip install tweepy`**


In [2]:
import tweepy

auth = tweepy.OAuthHandler("FHolBw3mVFGLtbfLb1m8zGVFV", "dIT8PT37qcpZkbaV1gnHfCiSHq60B9OmJnfxGQwwGyHacQ2n2T")
auth.set_access_token("1334257369027579907-CrBHS22plIxU4wpSHaRR1wtZdNuKYv", "Mga6h9zLXCZB53CPtDmbEAzprF8saZSI44Fis9ZfV7ooI")

api = tweepy.API(auth)


### 1.2   Extraction :

In [3]:
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
search_words = "news OR actualité"
new_search = search_words + " -filter:retweets"
date_since = "2020-12-08"
date_until = ""
totalTweets =2000
count = 1000
lang = "fr" or "en"
geocode = ""
result_type = "recent"
include_entities = True
filename = "testT3witter"

In [4]:
# Fonction de gestion de la pagination dans notre recherche
def limit_handled(cursor):
    while True:
        try:
            yield cursor.next()
        except tweepy.RateLimitError:
            print('Reached rate limite. Sleeping for >15 minutes')
            time.sleep(15 * 61)

In [5]:
# Fonction pour effectuer la recherche à l'aide de l'API Twitter
def search_tweets(new_search, date_since):

    # effectue la recherche à l'aide des variables définies
    for tweet in limit_handled(tweepy.Cursor(api.search,
                               q=new_search,
                               count=count,
                               tweet_mode='extended',
                               lang=lang,
                               geocode=geocode,
                               result_type=result_type,
                               include_entities=include_entities,
                               since=date_since,
                               until=date_until).items(totalTweets)):

        try:

            # Vérifie s'il s'agit d'un tweet étendu (> 140 caractères)
            content = tweet.full_text

            '''Convertir toutes les références de caractères nommés et numériques
             (par exemple & gt ;, & # 62 ;, & # x3e;) dans la chaîne s au
             caractères Unicode correspondants'''
            content = (content.replace('&amp;', '&').replace('&lt;', '<')
                       .replace('&gt;', '>').replace('&quot;', '"')
                       .replace('&#39;', "'").replace(';', " ")
                       .replace(r'\u', " ").replace('\u2026', ""))

            # Enregistrer d'autres informations du tweet
            user = tweet.author.screen_name
            timeTweet = tweet.created_at
            source = tweet.source
            tweetId = tweet.id
            tweetUrl = "https://twitter.com/statuses/" + str(tweetId)

            # Exclure les retweets, trop de mentions et trop de hashtags
            if not any((('RT @' in content, 'RT' in content,
                       content.count('@') >= 2, content.count('#') >= 3))):

                # Enregistre les informations du tweet dans une nouvelle ligne du fichier CSV
                writer.writerow([content, timeTweet,
                                user, source, tweetId, tweetUrl])

        except Exception as e:
            print('Encountered Exception:', e)
            pass


In [6]:
"""
import csv
from requests.exceptions import Timeout, ConnectionError
import ssl
from requests.packages.urllib3.exceptions import ReadTimeoutError
def work():

    # Ouverture d'un fichier CSV pour enregistrer les tweets rassemblés
    with open(filename+".csv", 'w') as file:
        global writer
        writer = csv.writer(file)

        # Ajouter une ligne d'en-tête au CSV
        writer.writerow(["Contenu_", "Date_", "Utilisateur_",
                         "Source_", "ID_", "Tweet URL"])

        # Initialisation de la recherche Twitter
        try:
            search_tweets(search_words, date_since)

        # Arrêtez temporairement lorsque vous atteignez la limite de taux Twitter
        except tweepy.RateLimitError:
            print("RateLimitError...waiting ~15 minutes to continue")
            time.sleep(1001)
            search_tweets(search_words, date_since)

        # Arrêtez temporairement en cas de dépassement de délai ou d'erreur de connexion
        except (Timeout, ssl.SSLError, ReadTimeoutError,
                ConnectionError) as exc:
            print("Timeout/connection error...waiting ~15 minutes to continue")
            time.sleep(1001)
            search_tweets(search_words, date_since)

        # Arrêtez temporairement lorsque vous recevez d'autres erreurs
        except tweepy.TweepError as e:
            if 'Failed to send request:' in e.reason:
                print("Time out error caught.")
                time.sleep(1001)
                search_tweets(search_words, date_since)
            elif'Too Many Requests' in e.reason:
                print("Too many requests, sleeping for 15 min")
                time.sleep(1001)
                search_tweets(search_words, date_since)
            else:
                print(e)
                print("Other error with this user...passing")
                pass


if __name__ == '__main__':

    work()
"""    

'\nimport csv\nfrom requests.exceptions import Timeout, ConnectionError\nimport ssl\nfrom requests.packages.urllib3.exceptions import ReadTimeoutError\ndef work():\n\n    # Ouverture d\'un fichier CSV pour enregistrer les tweets rassemblés\n    with open(filename+".csv", \'w\') as file:\n        global writer\n        writer = csv.writer(file)\n\n        # Ajouter une ligne d\'en-tête au CSV\n        writer.writerow(["Contenu_", "Date_", "Utilisateur_",\n                         "Source_", "ID_", "Tweet URL"])\n\n        # Initialisation de la recherche Twitter\n        try:\n            search_tweets(search_words, date_since)\n\n        # Arrêtez temporairement lorsque vous atteignez la limite de taux Twitter\n        except tweepy.RateLimitError:\n            print("RateLimitError...waiting ~15 minutes to continue")\n            time.sleep(1001)\n            search_tweets(search_words, date_since)\n\n        # Arrêtez temporairement en cas de dépassement de délai ou d\'erreur de conn

# Partie 2 : Prétraitement des tweets

il faut tout d'abord collecter les trois fichiers qu'on a créer : 
    
    extractionNews1Twitter.csv
    extractionNews2Twitter.csv
    extractionNews3Twitter.csv

## 2.1) Sauvgarder les tweets dans une dataframe :
### 2.1.1)  1ere dataframe ***03/12/2020***

In [7]:
import pandas as pd
données1 = pd.read_csv ('extractionNews1Twitter.csv',encoding= 'unicode_escape') 
données1.head(5)

,Contenu_,Date_,Utilisateur_,Source_,ID_,Tweet URL
0,White House communications director Alyssa Far...,2020-12-03 22:19:11,PTekach,Twitter for Android,1334623241416151040,https://twitter.com/statuses/1334623241416151040
1,Sen. Loeffler made large donations to anti-abo...,2020-12-03 22:19:09,chemasilva123,Twitter Web App,1334623233178558464,https://twitter.com/statuses/1334623233178558464
2,"Fake news, Barth na biso. Le jour viendra où c...",2020-12-03 22:19:05,Mutombolaurent,Twitter Web App,1334623218016063488,https://twitter.com/statuses/1334623218016063488
3,L'homme d'affaires était... à Barcelone\r\n\r\...,2020-12-03 22:18:40,YahooPopFR,TweetDeck,1334623111640125441,https://twitter.com/statuses/1334623111640125441
4,Consultation Développement soutenable 2020-202...,2020-12-03 22:17:16,LifeSciencesUPS,Scoop.it,1334622757632569344,https://twitter.com/statuses/1334622757632569344


In [8]:
print(données1.shape)

(3162, 6)


### 2.1.2) 2eme dataframe ***07/12/2020***

In [9]:
données2 = pd.read_csv ('extractionNews2Twitter.csv',encoding= 'unicode_escape') 
données2.head(5)

,Contenu_,Date_,Utilisateur_,Source_,ID_,Tweet URL
0,"Lesbians, man.\r\n\r\nhttps://t.co/3A86nx0FAA",2020-12-07 21:02:55,Civil_Beaver,Twitter for Android,1336053598992674817,https://twitter.com/statuses/1336053598992674817
1,"Actualités - AR - Après deux nouveaux crimes, ...",2020-12-07 21:02:36,ebenesport,ebenesport autopublish,1336053521184157696,https://twitter.com/statuses/1336053521184157696
2,Le président Tebboune serait en Algérie ce soi...,2020-12-07 21:02:29,Dmounir6,Twitter for Android,1336053488632131585,https://twitter.com/statuses/1336053488632131585
3,Actualités - AR - La Suède et la Grèce dans le...,2020-12-07 21:02:27,ebenesport,ebenesport autopublish,1336053483703775233,https://twitter.com/statuses/1336053483703775233
4,@Workyz_ Comment ça pour lutter contre les fak...,2020-12-07 21:02:24,blmawo,Twitter for Android,1336053470609149952,https://twitter.com/statuses/1336053470609149952


In [10]:
print(données2.shape)

(3237, 6)


### 2.1.3)   3eme dataframe ***11/12/2020***

In [11]:
import pandas as pd
données3 = pd.read_csv ('extractionNews3Twitter.csv',encoding= 'unicode_escape') 
données3.head(5)

,Contenu_,Date_,Utilisateur_,Source_,ID_,Tweet URL
0,Ford livre des repas en voitures autonomes à M...,2020-12-11 16:20:44,Alorscaroule,Twitter for Android,1337432137956548609,https://twitter.com/statuses/1337432137956548609
1,MERCATO: Limmense défi de Messi en Italie ? -...,2020-12-11 16:20:42,ebenesport,ebenesport autopublish,1337432130905845760,https://twitter.com/statuses/1337432130905845760
2,Le jeu de stratégie et de gestion en bac à sab...,2020-12-11 16:20:37,N_Difference,TweetDeck,1337432109619814402,https://twitter.com/statuses/1337432109619814402
3,Eddie Van Halen fait partie des personnes les ...,2020-12-11 16:20:24,MetalZoneFR,IFTTT,1337432055085461505,https://twitter.com/statuses/1337432055085461505
4,Votez pour le meilleur album Metal/Rock de l'a...,2020-12-11 16:20:20,MetalZoneFR,IFTTT,1337432035993022464,https://twitter.com/statuses/1337432035993022464


In [12]:
print(données3.shape)

(3125, 6)


###  2.1.4) Concatiner les trois dataFrames 

In [13]:
df_merge_col = pd.concat([données3,données1, données2 ])
print(df_merge_col.shape)

(9524, 6)


In [14]:
df_merge_col.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9524 entries, 0 to 3236
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Contenu_      9524 non-null   object
 1   Date_         9524 non-null   object
 2   Utilisateur_  9524 non-null   object
 3   Source_       9507 non-null   object
 4   ID_           9524 non-null   int64 
 5   Tweet URL     9524 non-null   object
dtypes: int64(1), object(5)
memory usage: 520.8+ KB


on va s'interesser juste sur la colonne **Contenu_**

## 2.2) Prétraitement des tweets

Dans cette étape, l’objectif est d’éliminer le texte inutile des tweets tels que les #, les noms des utilisateurs, les url, …

Le nettoyage des tweets comprendra plusieurs choses 

pour ne pas se tromper il vaut mieux aller du plus restrictif au moins restrictif.


### 2.2.1) Eliminer les données unitilies : nom-utilisateur, URL...

In [15]:
delete=["Tweet URL","ID_","Source_","Utilisateur_","Date_"]
alldata = df_merge_col.drop(delete, axis=1)
alldata.head(20)

,Contenu_
0,Ford livre des repas en voitures autonomes à M...
1,MERCATO: Limmense défi de Messi en Italie ? -...
2,Le jeu de stratégie et de gestion en bac à sab...
3,Eddie Van Halen fait partie des personnes les ...
4,Votez pour le meilleur album Metal/Rock de l'a...
5,Ella's Absence Seizures Explained by Ella  EP...
6,"The game awards, les annonces - Shady Part of ..."
7,"Cyberpunk 2077 : oups, des pénis sortent des p..."
8,J'aime beaucoup les frères #Tadros ( #DigitalW...
9,Persona 5 Strikers : On a vu la suite de Perso...


### 2.2.2) Eliminer les les caractéres non utiles : #, @ ...

Voilà à quoi ressemble notre pipeline :

In [16]:
import re

def nlp_pipeline(text):

    text = text.lower()
    text = text.replace('\n', ' ').replace('\r', '')
    text = ' '.join(text.split())
    text = re.sub(r"[A-Za-z\.]*[0-9]+[A-Za-z%°\.]*", "", text)
    text = re.sub(r"(\s\-\s|-$)", "", text)
    text = re.sub(r"[,\!\?\%\(\)\/\"]", "", text)
    text = re.sub(r"\&\S*\s", "", text)
    text = re.sub(r"\&", "", text)
    text = re.sub(r"\+", "", text)
    text = re.sub(r"\#", "", text)
    text = re.sub(r"\$", "", text)
    text = re.sub(r"\£", "", text)
    text = re.sub(r"\%", "", text)
    text = re.sub(r"\:", "", text)
    text = re.sub(r"\@", "", text)
    text = re.sub(r"\-", "", text)
    text = re.sub(r"\Ã", "", text)
    text = re.sub(r"\©", "", text)
    
    

    return text

Ce pipeline nous permet d’avoir des tweets à peu prés propres

on va l'appliquer sur la colonne`Contenu_`

In [17]:
corpus = alldata['Contenu_']
corpus_clean = corpus.apply(nlp_pipeline)

In [18]:
corpus_clean.head(20)

0     ford livre des repas en voitures autonomes à m...
1     mercato limmense défi de messi en italie http...
2     le jeu de stratégie et de gestion en bac à sab...
3     eddie van halen fait partie des personnes les ...
4     votez pour le meilleur album metalrock de l'an...
5     ella's absence seizures explained by ella  ep...
6     the game awards les annoncesshady part of me e...
7     cyberpunk   oups des pénis sortent des pantalo...
8     j'aime beaucoup les frères tadros  digitalworl...
9     persona  strikers  on a vu la suite de persona...
10    felicity machado est ma  ième source d'actuali...
11    un tournevis semble l'outil le plus simple au ...
12    un sénateur républicain blanc et solitaire blo...
13    emmanuelmacron violencespolicières manifestant...
14                         viruswar merci pour la news 
15              donc ce n'est jamais arrivé.. fake news
16    rclens rclmhsc | lensmontpellier le groupe len...
17    l'ia pourrait remplacer l'homme dans le se

### 2.2.3) sauvgarder les donner dans un fichier csv 

In [19]:
corpus_clean.to_csv("corpus_clean.csv",index=False, encoding='utf-8')

In [20]:
import pandas as pd
df = pd.read_csv ('corpus_clean.csv',encoding= 'utf-8') 
df.head(20)

,Contenu_
0,ford livre des repas en voitures autonomes à m...
1,mercato limmense défi de messi en italie http...
2,le jeu de stratégie et de gestion en bac à sab...
3,eddie van halen fait partie des personnes les ...
4,votez pour le meilleur album metalrock de l'an...
5,ella's absence seizures explained by ella  ep...
6,the game awards les annoncesshady part of me e...
7,cyberpunk oups des pénis sortent des pantalo...
8,j'aime beaucoup les frères tadros digitalworl...
9,persona strikers on a vu la suite de persona...


In [21]:
import numpy as np
array=np.array(df)
import pandas as pd 
dataframe=pd.DataFrame(array,columns=['Contenu_'])

# Partie 3 : Traitement des tweets : NLP (Natural LanguageProcessing)

## **Install NLTK**

`!pip3 install nltk`

In [22]:
import nltk

In [23]:
#nltk.download()

`nltk.download()`

## 3.1) Segmentation

> on peut appliquer la segmentation mes ca sert à rien puisque en utilie une Dataframe

In [24]:
from nltk.tokenize import sent_tokenize
with open("corpus_clean.csv") as f:
    s=f.read()
sents=(sent_tokenize(s))
#print(sents)

## 3.2) Tokenization

### extracter tous les mots 

> on peut extracter tous les mots comme indique le code suivant 

In [25]:
from nltk.tokenize import word_tokenize
import re

In [26]:
#from nltk.tokenize import word_tokenize
#with open("corpus_clean.csv") as f:
#    s=f.read()
#words=word_tokenize(s)

#print(words)

#print (words[50])

mais on a besoins de tokenization de chaque tweet comme suivant : 

### transformer chaque tweet en un ensemble de mots

In [27]:
df_token=dataframe
df_token['Contenu_']=df_token['Contenu_'].apply(word_tokenize)

In [28]:
print(df_token.shape)

(9524, 1)


In [29]:
df_token.head(10)

,Contenu_
0,"[ford, livre, des, repas, en, voitures, autono..."
1,"[mercato, limmense, défi, de, messi, en, ital..."
2,"[le, jeu, de, stratégie, et, de, gestion, en, ..."
3,"[eddie, van, halen, fait, partie, des, personn..."
4,"[votez, pour, le, meilleur, album, metalrock, ..."
5,"[ella, 's, absence, seizures, explained, by, e..."
6,"[the, game, awards, les, annoncesshady, part, ..."
7,"[cyberpunk, oups, des, pénis, sortent, des, pa..."
8,"[j'aime, beaucoup, les, frères, tadros, digita..."
9,"[persona, strikers, on, a, vu, la, suite, de, ..."


 > notre data frame est transformée à une data frame des listes 

## 3.3) lemmatisation 

In [30]:
import nltk

`nltk.download()`

In [31]:
from nltk.stem import wordnet
from nltk.stem import WordNetLemmatizer
word_lem=WordNetLemmatizer()
for word in words:
    print(word +":" + word_lem.lemmatize(word))

NameError: name 'words' is not defined

### 3.4) Stemming

`nltk.download('stopwords')`

In [ ]:
import re
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
ps= PorterStemmer()
stemmed_dataset=[]
for i in range(0,9524):
    stemmed_array = df['Contenu_'][i].split()
    stemmed = [ps.stem(word) for word in stemmed_array if not word in set(stopwords.words('english'))]
    stemmd =' '.join(stemmed)
    stemmed_dataset.append(stemmed)
print(stemmed_dataset[0:5])

In [ ]:
df_stem = pd.DataFrame(stemmed_dataset)
df_stem.head()

In [ ]:
print(df_stem.shape)

# 4) Clustering

## 1. Convertissez des phrases en vecteurs en utilisant TFIDF

Tout d'abord, nous devrons créer un modèle gensim pour convertir nos données textuelles en représentation vectorielle. 

Transforme le texte en une matrice clairsemée 

In [ ]:
corpus_clean.head(20)
print(corpus_clean.shape)

In [ ]:
text=(corpus_clean.to_string())
#print(text)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
counts = count_vect.fit_transform(corpus_clean)
counts.shape

On aboutit à une matrice sparse ou chaque expression est représentée à une vecteur ou chaque 1 représente l’appartenance d’un mot à l’ensemble.

In [ ]:
type(counts)

In [ ]:
#print(counts)

### td-idf

Effectue la transformation TF-IDF à partir d'une matrice de comptage fournie.

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer()
res = tfidf.fit_transform(counts)
res.shape

In [ ]:
print(res)

In [ ]:
#res[0,:].sum()

**************************************************************************************************************

In [ ]:

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
res2 = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names())
print(res2)


In [ ]:
print(res2.shape)

In [ ]:
from sklearn.cluster import KMeans
wcss=[]

for i in range(0,df.shape[0]):
    kmeans =KMeans(n_clusters=3)
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, init='k-means++', n_init=10, max_iter=300, random_state=0, verbose=True)
    kmeans.fit(res)
    wcss.append(kmeans.inertia_)
    

